In [3]:
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
import cv2
import requests

# API configuration
API_KEY = "###"  # Replace this with your Perenual API key
API_URL = "https://perenual.com/api/v2/plants"

# Create the main window
root = tk.Tk()
root.title("Plant Indicator App")

# Create a notebook (tabs) widget
notebook = ttk.Notebook(root)

# Create two frames (Photo Recognition and Plant Information)
frame1 = ttk.Frame(notebook)
frame2 = ttk.Frame(notebook)

# Add the frames to the notebook
notebook.add(frame1, text="Photo Recognition")
notebook.add(frame2, text="Plant Information")
notebook.pack(expand=1, fill="both")

# Frame 1: Photo Recognition
photo_label = ttk.Label(frame1, text="Upload a Plant Image:")
photo_label.grid(row=0, column=0, padx=10, pady=10)

# Label to display the selected image
image_label = tk.Label(frame1)
image_label.grid(row=1, column=0, padx=10, pady=10, columnspan=2)

plant_name_label = ttk.Label(frame1, text="")  # Initially empty
plant_name_label.grid(row=2, column=0, padx=10, pady=10, columnspan=2)# Frame 1: Photo Recognition
photo_label = ttk.Label(frame1, text="Upload a Plant Image:")
photo_label.grid(row=0, column=0, padx=10, pady=10)

# Label to display the selected image
image_label = tk.Label(frame1)
image_label.grid(row=1, column=0, padx=10, pady=10, columnspan=2)

# Label to display the plant name below the image
plant_name_label = ttk.Label(frame1, text="")  # Initially empty
plant_name_label.grid(row=2, column=0, padx=10, pady=10, columnspan=2)

def open_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        # Open the image with Pillow and display it
        img = Image.open(file_path)
        img.thumbnail((300, 300))  # Resize for display purposes
        img_tk = ImageTk.PhotoImage(img)
        image_label.config(image=img_tk)
        image_label.image = img_tk  # Keep a reference

        # Recognize the plant from the image
        recognize_plant(file_path)

upload_button = ttk.Button(frame1, text="Upload Image", command=open_image)
upload_button.grid(row=0, column=1, padx=10, pady=10)

# Function to call the Perenual API and recognize the plant
def recognize_plant(image_path):
    # Open the image file in binary mode
    with open(image_path, "rb") as image_file:
        # Prepare the API request
        files = {"image": image_file}
        params = {"api_key": API_KEY}

        # Send the request to Perenual API
        response = requests.post(f"{API_URL}/identify", files=files, params=params)

        if response.status_code == 200:
            data = response.json()
            if data and data.get('data'):
                # Extract plant info from the response
                plant_name = data['data'][0]['common_name']  # Example field: common name
                scientific_name = data['data'][0]['scientific_name']
                care_level = data['data'][0].get('care_level', 'Not available')
                light_level = data['data'][0].get('light', 'Not available')

                # Display the plant information in the Plant Information frame
                plant_info = f"Plant Name: {plant_name}\nScientific Name: {scientific_name}\nCare Level: {care_level}\nLight Level: {light_level}"
                plant_info_label.config(text=plant_info)

                # Update the plant name label under the image
                plant_name_label.config(text=f"Plant Name: {plant_name}")
            else:
                plant_info_label.config(text="No plant recognized.")
                plant_name_label.config(text="No plant recognized.")
        else:
            plant_info_label.config(text=f"Error: {response.status_code}")
            plant_name_label.config(text="Recognition failed.")

# Frame 2: Plant Information
plant_info_label = ttk.Label(frame2, text="Plant information will appear here after photo recognition.")
plant_info_label.grid(row=0, column=0, padx=10, pady=10)

# Run the application
root.mainloop()